#### 1. Расскажите о том как работает регуляризация в решающих деревья, какие параметры мы штрафуем в данных алгоритмах?

$\textbf{Регуляризация}$ в статистике, машинном обучении, теории обратных задач — метод добавления некоторых дополнительных ограничений к условию с целью решить некорректно поставленную задачу или предотвратить переобучение. Эта информация часто имеет вид штрафа за сложность модели. Например, это могут быть ограничения гладкости результирующей функции или ограничения по норме векторного пространства.<br>
Ошибка, минимизируемая в градиентном бустинге: $err(h) = \sum_{j=1}^N L(\sum_{i=1}^{T-1}a_ib_i(x_j) + b_Ta_T(x_j))$<br>
Добавим регуляризацию в этот функционал ошибки: $err(h) = \sum_{j=1}^N L(\sum_{i=1}^{T-1}a_ib_i(x_j) + b_Ta_T(x_j)) + \sum_{i=1}^T \omega(a_i)$<br>
Регрессия с регуляризацией построена по принципу: чем больше по норме значения коэффициентов перед регрессией, тем хуже, поэтому нужно, чтобы они были как можно меньше.<br>
В деревьях идея очень похожа: дерево штрафуется, если сумма нормы значений в листьях в нём очень большая.<br>
Всё сводится к тому, что новое разбиение мы будем выбирать, максимизируя gain: $Gain = \frac{G_l^2}{S_l^2 + \lambda} + \frac{G_r^2}{S_r^2 + \lambda} - \frac{(G_l + G_r)^2}{S_l^2 + S_r^2 + \lambda} - \gamma$<br>
Здесь $\gamma, \lambda$ — это числовые параметры регуляризации, а $G_i, S_i$ — соответствующие суммы из первых и вторых производных при данном разбиении.<br>

Ссылки на источники:
1. https://ru.wikipedia.org/wiki/Регуляризация_(математика)
2. https://habr.com/ru/company/mailru/blog/438562/
3. https://moluch.ru/archive/131/36581/

#### 2. По какому принципу рассчитывается "важность признака (feature_importance)" в ансаблях деревьев?

Если построить много деревьев решений (случайный лес), то чем выше в среднем признак в дереве решений, тем он важнее в данной задаче классификации/регрессии. При каждом разбиении в каждом дереве улучшение критерия разделения (например неопределенность Джини(Gini impurity)) — это показатель важности, связанный с переменной разделения, и накапливается он по всем деревьям леса отдельно для каждой переменной.<br>

Среднее снижение точности, вызываемое переменной, определяется во время фазы вычисления out-of-bag ошибки. Чем больше уменьшается точность предсказаний из-за исключения (или перестановки) одной переменной, тем важнее эта переменная, и поэтому переменные с бо́льшим средним уменьшением точности более важны для классификации данных. Среднее уменьшение неопределенности Джини (или ошибки mse в задачах регрессии) является мерой того, как каждая переменная способствует однородности узлов и листьев в окончательной модели случайного леса. Каждый раз, когда отдельная переменная используется для разбиения узла, неопределенность Джини для дочерних узлов рассчитывается и сравнивается с коэффициентом исходного узла. Неопределенность Джини является мерой однородности от 0 (однородной) до 1 (гетерогенной). Изменения в значении критерия разделения суммируются для каждой переменной и нормируются в конце вычисления. Переменные, которые приводят к узлам с более высокой чистотой, имеют более высокое снижение коэффициента Джини.<br>

В стеке sklearn, в пакетах xgboost, lightGBM есть встроенные методы оценки важности фичей (feature importance) для «деревянных моделей»:
1. $\textbf{Gain}$<br>
Эта мера показывает относительный вклад каждой фичи в модель. для расчета мы идем по каждому дереву, смотрим в каждом узле дерева какая фича приводит к разбиению узла и насколько снижаетcя неопределенность модели согласно метрике (Gini impurity, information gain).
Для каждой фичи суммируется её вклад по всем деревьям.
2. $\textbf{Cover}$<br>
Показывает количество наблюдений для каждой фичи. Например, у вас 4 фичи, 3 дерева. Предположим, фича 1 в узлах дерева содержит 10, 5 и 2 наблюдения в деревьях 1, 2 и 3 соответственно Тогда для данной фичи важность будет равна 17 (10 + 5 + 2).
3. $\textbf{Frequency}$<br>
Показывает, как часто данная фича встречается в узлах дерева, то есть считается суммарное количество разбиений дерева на узлы для каждой фичи в каждом дереве.

В библиотеке $\textbf{SHAP}$ для оценки важности фичей рассчитываются значения Шэпли (по имени американского математика и названа библиотека). Для оценки важности фичи происходит оценка предсказаний модели с и без данной фичи.<br>

Ссылки на источники: 
1. https://habr.com/ru/company/ods/blog/324402/#3-ocenka-vazhnosti-priznakov
2. https://habr.com/ru/post/428213/
3. https://shap.readthedocs.io/en/latest/
4. https://github.com/slundberg/shap 